# Réalisation d'un gradient de température au dessus de la carotte EPICA DOME C

In [ ]:
import xarray as xr
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.util import add_cyclic_point
import intake, intake_esm
import os
import sys
from pathlib import Path
from glob import glob
import itertools
import time
import seaborn as sns
import math
import scipy.stats as st
import requests

import warnings
warnings.filterwarnings('ignore')

%run functions.ipynb

In [ ]:
'''
Les fichiers sont stockés dans deux catalogues :
    - PMIPCat.json : Modèles Paleo
    - CMIPCat.json : Modèles Actuel (avec piControl notamment)
'''
PMIP = intake.open_esm_datastore("PMIPCat.json")

In [ ]:
CMIP = intake.open_esm_datastore('CMIPCat.json')

## Dask command

In [ ]:
'''
Pour permettre de faire tourner le prochain script il est nécessaire de demander 
de la mémoire de stockage et des coeurs de travails plus important que ceux
fournis pour le JupyterLab (14Go). On utilise donc dask. 

Ce script est tiré du Jupyter notebook de Laurent Troussellier. Pour comprendre
les détails voir :

    https://ipsl.pages.in2p3.fr/projets/paleo-ipsl/modelobsnotebooks/NBExplicatif/CMIP6Projection.html

'''

from dask.distributed import Client, progress
from dask_jobqueue import PBSCluster
import time


# Per job
nb_cores = 2
mem = 12 # in GiB (not GB).
vmem = mem # in GiB # Swapping space.
walltime = '04:00:00'

nb_jobs = 4
network_interface = 'ib0'

# If the port 8787 is already binded, this instruction will output the actual port.
cluster = PBSCluster(cores=nb_cores, memory=f'{mem}GiB', interface=network_interface, processes=nb_cores, walltime=walltime,
                     job_extra=(f'-l mem={mem}gb', f'-l vmem={vmem}gb', f'-l nodes=1:ppn={nb_cores}'))



print(f'the dashboard link: {cluster.dashboard_link}')




cluster.scale(jobs=nb_jobs) 
client = Client(cluster)
client.wait_for_workers(n_workers=nb_jobs) # This instruction blocks until the job are almost ready.
# May be wait more for the job to be ready (running).
print('almost there')

time.sleep(60)

print("ready to work")

## Data acquisition

In [ ]:
'''
EPICA Dome C Coordinates
'''

lat_edc = -75.06
lon_edc = 123.2

In [ ]:
''' 
L'objectif est de comparer la variance d'une variable entre périodes.

Plusieurs étapes :
    - spécification des variables
    - création d'un dataframe vide qui va être rempli
    - import des données via le catalogue pour le piControl et pour les autres périodes. 
      Si le chemin existe pour chaque variables alors le fichier est téléchargé. 
      Durant l'import, les données sont :
            . aggrégées avec dask
            . réduite à la zone autours d'EPICA Dome C
            . moyennées dans l'espace
            . On récupère la variance dans la temps
            
    - enregistrement dans le dataframe : df
'''

var = 'ta'
inst = PMIP.df['institution_id'].unique()
source = PMIP.df['source_id'].unique()
date = [
    'lgm',
    'lig127k'
    ]
grid = [
    'gn', 
    'gr1', 
    'gr'
    ]
res_lat = 1.25
res_lon = 1.75

df = pd.DataFrame(
    columns = [
        'ta',
        'plev',
        'institute',
        'source',
        'period',
        'grid'
        ])

for i in inst :
    for s in source :
        for g in grid :
            if os.path.exists(
                '/bdd/CMIP6/CMIP/{}/{}/piControl/r1i1p1f1/Amon/{}/{}/latest'.format(i, s, var, g)) :
                # piControl
                X = CMIP.search(
                    institution_id= '{}'.format(i), 
                    variable_id = var,
                    source_id = '{}'.format(s),
                    table_id = 'Amon', 
                    experiment_id= 'piControl',  
                    latest = True,  
                    member_id = "r1i1p1f1",
                    grid_label = '{}'.format(g)
                    )
                ds_pi = xr.open_mfdataset(
                    list(X.df["path"]),
                    chunks = 100,
                    use_cftime=True,
                    decode_cf=True
                    ).ta[:1000]
                ds_pi = ds_pi.where((
                    (ds_pi["lat"] > lat_edc - 2.5) 
                    & (ds_pi['lat'] < lat_edc + 2.5) 
                    & (ds_pi['lon'] > lon_edc - 5) 
                    & (ds_pi['lon'] < lon_edc + 5)),
                    drop=True
                    )
                ds_pi = ds_pi.mean(['time', 'lat', 'lon'])

                print('Computing piControl for : {} - {} :'.format(s, var))
                with ProgressBar():
                    ds_pi = ds_pi.compute()
                    
                data = pd.DataFrame()
                data['ta'] = pd.Series(ds_pi.values)
                data['plev'] = pd.Series(ds_pi.plev.values)
                data['institute'] = i
                data['source'] = s
                data['period'] = 'piControl'
                data['grid'] = g
                df = df.append(data)
                
                
            for d in date :
                if os.path.exists('/bdd/CMIP6/PMIP/{}/{}/{}/r1i1p1f1/Amon/{}/{}/latest'.format(i, s, d, var, g)):
                    # Period 
                    X = PMIP.search(
                        institution_id= '{}'.format(i), 
                        variable_id = var,
                        source_id = '{}'.format(s),
                        table_id = 'Amon', 
                        experiment_id= '{}'.format(d),  
                        latest = True,  
                        member_id = "r1i1p1f1",
                        grid_label = '{}'.format(g)
                        )
                    ds_old = xr.open_mfdataset(
                        list(X.df["path"]),
                        chunks = 100,
                        use_cftime=True,
                        decode_cf=True
                        )
                    ds_old = ds_old.where((
                        (ds_old.coords["lat"] > lat_edc - 2.5) 
                        & (ds_old.coords['lat'] < lat_edc + 2.5) 
                        & (ds_old.coords['lon'] > lon_edc - 5) 
                        & (ds_old.coords['lon'] < lon_edc + 5)),
                        drop=True
                        )
                    ds_old = ds_old.mean(['time', 'lat', 'lon'])
                    '''
                    ds_old = regrid(
                        ds_old, 
                        res_lat, 
                        res_lon, 
                        -90, 90, 0, 360
                        )
                    ds_old = ds_old.where((
                        (ds_old.coords["lat"] > lat_edc - 1 )
                            & (ds_old.coords['lat'] < lat_edc + 1) 
                            & (ds_old.coords['lon'] > lon_edc - 2) 
                            & (ds_old.coords['lon'] < lon_edc + 2)),
                        drop=True
                        ) 
                    '''

                    print('Computing {} for : {} - {} :'.format(d, s, var))
                    with ProgressBar():
                        ds_old = ds_old.compute()                      

                    data = pd.DataFrame()
                    data['ta'] = pd.Series(np.array(ds_old.ta))
                    data['plev'] = pd.Series(np.array(ds_old.plev))
                    data['institute'] = i
                    data['source'] = s
                    data['period'] = d
                    data['grid'] = g
                    df = df.append(data)

In [ ]:
var = 'ta'
inst = PMIP.df['institution_id'].unique()
source = PMIP.df['source_id'].unique()
date = [
    'lgm',
    'lig127k'
    ]
grid = [
    'gn', 
    'gr1', 
    'gr'
    ]
res_lat = 1.25
res_lon = 1.75

df = pd.DataFrame(
    columns = [
        'ta',
        'plev',
        'institute',
        'source',
        'period',
        'grid'
        ])

for s in source :
    for i in inst :
        for g in grid :
        # Pre-industrial Control
            if os.path.exists(
                '/bdd/CMIP6/CMIP/{}/{}/piControl/r1i1p1f1/Amon/{}/{}/latest'.format(i, s, var, g)) :
                Y = CMIP.search(
                    institution_id = '{}'.format(i), 
                    variable_id = var,
                    source_id = '{}'.format(s),
                    table_id = 'Amon', 
                    experiment_id = 'piControl',
                    latest = True, 
                    member_id = 'r1i1p1f1'
                    )
                for l in range(len(Y.df)):                
                    ds_pi = xr.open_dataset(
                        Y.df['path'][l], 
                        use_cftime = True, 
                        decode_cf = True,
                        chunks = 100, 
                        drop_variables = [
                            'lat_bnds', 'lon_bnds', 'time_bnds', 
                            'i', 'j', 
                            'vertices_longitude', 'vertices_latitude'
                            ]).ta[:1000]
                    ds_pi = ds_pi.mean(['time', 'lat', 'lon'])

                    print('Computing piControl for : {} - {} :'.format(s, var))
                    with ProgressBar():
                        ds_pi = ds_pi.compute()
                    
                    data = pd.DataFrame()
                    data['ta'] = pd.Series(ds_pi.values)
                    data['plev'] = pd.Series(ds_pi.plev.values)
                    data['institute'] = i
                    data['source'] = s
                    data['period'] = 'piControl'
                    data['grid'] = g
                    df = df.append(data)
                
            for d in date :
                if os.path.exists(
                    '/bdd/CMIP6/PMIP/{}/{}/{}/r1i1p1f1/Amon/{}/{}/latest'.format(i, s, d, var, g)):
                    # Period 
                    X = PMIP.search(
                        institution_id= '{}'.format(i), 
                        variable_id = var,
                        source_id = '{}'.format(s),
                        table_id = 'Amon', 
                        experiment_id= '{}'.format(d),  
                        latest = True,  
                        member_id = "r1i1p1f1",
                        grid_label = '{}'.format(g)
                        )
                    ds_old = xr.open_mfdataset(
                        list(X.df["path"]),
                        chunks = 500,
                        use_cftime=True,
                        decode_cf=True
                        )
                    ds_old = ds_old.mean(['time', 'lat', 'lon'])
                    '''
                    ds_old = regrid(
                        ds_old, 
                        res_lat, 
                        res_lon, 
                        -90, 90, 0, 360
                        )
                    ds_old = ds_old.where((
                        (ds_old.coords["lat"] > lat_edc - 1 )
                            & (ds_old.coords['lat'] < lat_edc + 1) 
                            & (ds_old.coords['lon'] > lon_edc - 2) 
                            & (ds_old.coords['lon'] < lon_edc + 2)),
                        drop=True
                        ) 
                    '''

                    print('Computing {} for : {} - {} :'.format(d, s, var))
                    with ProgressBar():
                        ds_old = ds_old.compute()                      

                    data = pd.DataFrame()
                    data['ta'] = pd.Series(np.array(ds_old.ta))
                    data['plev'] = pd.Series(np.array(ds_old.plev))
                    data['institute'] = i
                    data['source'] = s
                    data['period'] = d
                    data['grid'] = g
                    df = df.append(data)

In [ ]:
'''
Le travail ayant été fastidieux et a demandé du stockage externe,
il est conseillé d'enregistrer les données via la commande suivante.
'''

df.to_csv(
    r'/home/bchaigneau/Stage_LSCE/datas/df_grad_ta', 
    header=True, 
    index=None, 
    sep=' ', 
    mode='w'
    )

In [ ]:
'''
Pour lire les données précédemment entregistrées.
'''

df = pd.read_csv(
    '/home/bchaigneau/Stage_LSCE/datas/df_grad_ta', 
    sep = ' ', 
    names = ['ta', 'plev', 'institute', 'source', 'period', 'grid']
    )

## Plot du Gradient de Température

In [ ]:
'''
A partir des données aggrégées il est possible de représenter
le gradient de température autours du site EDC.

Sélection des données physiquement existantes (<70 hPa)

Les étapes :
    - mise en place de la structure du graph
    - sous échantillonnage par modèle puis plot
    - l'axe y est inversé
    - sauvegarde de la figure
'''



df = df.reset_index(drop = True)

a = df[df['plev'] <= 71000]

var = 'ta'

colors = {
    'piControl':'peru', 
    'lgm':'royalblue', 
    'lig127k':'forestgreen'
    }

sns.set_context("talk")
sns.set_style('ticks')
sns.set_context("talk") 
sns.set_style(
    'ticks', 
    {"grid.color":"0.9", 
     "grid.linestyle":":", 
     "axes.grid": True
    })

for s in a['source'].unique():   
    fig, ax = plt.subplots(figsize = (12, 8))
    x = a[a['source'] == s]
    for p in x.period.unique() :
        b = x[x['period'] == p]
        plt.plot(
            b.ta, b.plev, 
            c = colors[p],
            label = p
            )
        
    plt.ylim(
        max(a['plev']), 
        min(a['plev'] - 1000)
        )
    plt.xlim(
        np.nanmin(a['ta']),
        np.nanmax(a['ta'])
        )
    plt.title(
    'Distribution of the Temperature above the EPICA Dome C station\n\nModel : {}'.format(s), 
    size = 22,
    loc = 'left')
    plt.text(290, 85000, 
             'The data is the mean temperature at each level for an area of 10° lat and 30° lon',
             fontstyle = 'italic',
             horizontalalignment = 'right',
             fontsize = 15)
    plt.legend()
    plt.savefig('/home/bchaigneau/Stage_LSCE/plot/gradient_edc/{}_{}.jpeg'.format(s, var),dpi= 100)
    plt.show()

## Recuperation du dTc/dTs

In [ ]:
'''
Redimensionnement des données pour obtenir
la différence : piControl - lgm, pour chaque modèle.

Les étapes : 
    - Sélection des données associée que à 60 et 70 hPa, et en dehors du lig127ka
    - Réarrangement du tableau : pivot_table
    - Supprime toutes les colonnes qui possèdent un Nan (càd qu'il n'y pas de quoi faire la différence) : dropna
    - Réarrangement du tableau pour faire la différence en automatique
'''

b = pd.pivot_table(
    a[(a['period'] != 'lig127k') 
      & ((a['plev'] == 70000) | (a['plev'] == 60000))],            
    columns = 'source',
    values = 'ta', 
    index = ['period', 'plev']
    ).dropna(axis = 1)

c = pd.pivot_table(b, index = 'plev', aggfunc = 'diff').reset_index()
c

In [ ]:
'''
Création d'un tableau final qui fait le rapport de
la température à 70hPa/ 60hPa.
'''

rapport = pd.DataFrame()

for i in b.columns :
    rapport[i] = pd.Series(
        c[i][0] / c[i][1]
        )   

rapport = rapport.transpose().rename(columns = {0:'dTc/dTs'})
rapport

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
plt.scatter(rapport.index, rapport['dTc/dTs'])
plt.title(
    "Rapport de la température entre\ndTc/dTs (60hPa/70hPa) des différents modèles",
    loc = 'left'
    )
plt.savefig(
    '/home/bchaigneau/Stage_LSCE/plot/gradient_edc/inversion/rapport_inversion.jpeg',
    dpi= 100
    )